In [1]:
# 10/19/2021
# No need to run first section again. 
# Run import statements then proceed to Preprocess Data section where csv is loaded in.

# BINNING NEEDS WORK IN ORDER FOR MODELS TO RUN

In [2]:
# import dependencies
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
# from config import Password, Username, Host,DB
import pandas as pd

# Preprocess Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
movies_df = pd.read_csv("/content/drive/MyDrive/final project/dummy_data_frame.csv")

In [5]:
movie_id_list = movies_df["movie_id"].to_list()

In [6]:
# separate out target: Box Office
# bin target data
!pip install jenkspy
import jenkspy
labels = ["0-10724389", "10724389-27367660.0", "27367660-46462469","46462469-67790117","67790117-92186262","92186262-120146040", 
            "120146040-153636354", "153636354-191719337", "191719337-228778661", "228778661-268488329","268488329-312433331", 
            "312433331-368065385", "368065385-459005868", "459005868-533720947", "533720947-760507625"]
breaks = jenkspy.jenks_breaks(movies_df["boxoffice"], nb_class=15)
target = pd.cut(movies_df["boxoffice"] , bins=breaks, labels=labels, include_lowest=True)


     |████████████████████████████████| 55 kB 621 kB/s 
  Created wheel for jenkspy: filename=jenkspy-0.2.0-cp37-cp37m-linux_x86_64.whl size=70405 sha256=bc987ebd0157de2e94440426ca654828d796f4db92e01fe04337fd338d3af463
  Stored in directory: /root/.cache/pip/wheels/d8/d1/42/9bd05164ef706b84674905892f0542f8db6ac4a652473ca7e0
Successfully built jenkspy


In [7]:

movies_df.drop(columns=["boxoffice"], inplace=True)

In [8]:
# clean data
# date into month
movies_df["month"] = pd.DatetimeIndex(movies_df['date']).month


In [9]:
# make dummies for rating
dummy_df = pd.get_dummies(movies_df, columns=["rating"])

In [10]:
dummy_df.drop(columns=["movie_id", "date"], inplace=True)

In [11]:
dummy_df

,runtime,metascore,imdb_rating,budget,buzz,Shawn LevyDirector,Robert Ben GarantWriter,Thomas LennonWriter,Milan TrencWriter,Ben StillerActor,Carla GuginoActor,Ricky GervaisActor,Adventure,Comedy,Family,John LasseterDirector,John LasseterWriter,Joe RanftDirector,Joe RanftWriter,Jorgen KlubienWriter,Owen WilsonActor,Bonnie HuntActor,Paul NewmanActor,Animation,Brett RatnerDirector,Simon KinbergWriter,Zak PennWriter,Patrick StewartActor,Hugh JackmanActor,Halle BerryActor,Action,Sci-Fi,Ron HowardDirector,Akiva GoldsmanWriter,Dan BrownWriter,Tom HanksActor,Audrey TautouActor,Jean RenoActor,Mystery,Thriller,...,Reid MillerActor,Mark O'BrienActor,Justin ChonDirector,Justin ChonWriter,Justin ChonActor,Kevin LewisDirector,G.O. ParsonsWriter,Emily TostaActor,Beth GrantActor,Robert ConnollyDirector,Robert ConnollyWriter,Harry CrippsWriter,Jane HarperWriter,Genevieve O'ReillyActor,Quoc Bao TranDirector,Quoc Bao TranWriter,Andy LeActor,Ron YuanActor,Yuji OkumotoActor,Aidan ChambersWriter,Félix LefebvreActor,Benjamin VoisinActor,Philippine VelgeActor,Anthony Scott BurnsDirector,Anthony Scott BurnsWriter,Daniel WeissenbergerWriter,Julia Sarah StoneActor,Landon LiboironActor,Carlee RyskiActor,month,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R,rating_TV-14,rating_TV-MA,rating_TV-PG,rating_Unrated
0,108,48,6.4,110000000,6,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,1,0,0,0,0,0,0
1,117,73,7.1,70000000,4,0,0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,1,0,0,0,0,0,0,0,0,0
2,104,58,6.7,210000000,23,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,1,0,0,0,0,0
3,149,46,6.6,125000000,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,1,0,0,0,0,0
4,154,72,6.0,232000000,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,88,44,5.5,750000,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0
1663,117,69,6.9,1400000,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,1,0,0,0,0
1664,108,67,6.4,15000000,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,1,0,0,0,0,0
1665,101,65,6.9,7000000,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,6,0,0,1,0,0,0,0,0,0,0


In [12]:
# split data into test train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dummy_df, target, random_state=42)

In [13]:
# scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create Models

In [14]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=42, n_estimators=1000).fit(X_train_scaled, y_train)

In [15]:
# Training and testing scores
print(f'Training Score: {random_forest_model.score(X_train_scaled, y_train)}')
print(f'Testing Score: {random_forest_model.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.36930455635491605


In [16]:
# random forest feature importances

features = random_forest_model.feature_importances_
importance_df = pd.DataFrame({"ids": dummy_df.columns, "importances": features})
sorted_df = importance_df.sort_values(by="importances", ascending=False)
sorted_df.head(10)

,ids,importances
3,budget,0.085409
0,runtime,0.018757
1,metascore,0.018041
2,imdb_rating,0.017874
4,buzz,0.016430
7145,month,0.014974
75,Drama,0.004893
13,Comedy,0.004576
7150,rating_PG-13,0.004536
7151,rating_R,0.004321


In [17]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [18]:
random_forest_pca = RandomForestClassifier(random_state=42, n_estimators=1000).fit(X_train_pca, y_train)# Training and testing scores
print(f'Training Score: {random_forest_pca.score(X_train_pca, y_train)}')
print(f'Testing Score: {random_forest_pca.score(X_test_pca, y_test)}')

Training Score: 1.0
Testing Score: 0.3069544364508393


In [22]:
y_pred = random_forest_model.predict(X_test_scaled)

In [23]:
prediction_df = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
prediction_df

,Actual,Predicted
940,46462469-67790117,0-10724389
987,10724389-27367660.0,0-10724389
405,46462469-67790117,10724389-27367660.0
721,92186262-120146040,0-10724389
1612,10724389-27367660.0,0-10724389
...,...,...
1507,0-10724389,0-10724389
1395,120146040-153636354,27367660-46462469
292,10724389-27367660.0,0-10724389
1466,10724389-27367660.0,0-10724389


# Output Models

In [20]:
import pickle
pickle.dump(random_forest_model, open("/content/drive/MyDrive/final project/random_forest.pkl", 'wb'))